In [1]:
import sys
sys.path.append('/users/gxb18167/EEG-NLP')

from src import data
from src import utils
from src import Networks

import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [3]:
d = data.Data()
util = utils.Utils()

In [4]:
train_NE, train_EEG_segments, train_Classes = d.NER_read_custom_files(train_path)

In [5]:
train_NE[0]

['Henry', 'Ford']

In [6]:
vector_size = 50
#parameters['vector_size'] = vector_size
window = 5
#parameters['window'] = window
min_count = 1
#parameters['min_count'] = min_count
workers = 4

In [7]:
train_word_embeddings, train_NE_embedded = util.NER_Word2Vec(train_NE, vector_size, window, min_count, workers)

In [8]:
len(train_NE_embedded)

585

In [9]:
train_NE_expanded = util.NER_expanded_NER_list(train_EEG_segments, train_NE_embedded)

In [10]:
train_NE_expanded[0][0].shape

(50,)

In [37]:
max_seq_len = max([len(i) for i in train_NE_expanded])
#padding function
for i in range(len(train_NE_expanded)):
    padding_count = max_seq_len - len(train_NE_expanded[i])
    for j in range(padding_count):
        train_NE_expanded[i].append(np.zeros(vector_size))





In [12]:
import torch
from transformers import BertTokenizer, BertModel

class NER_BERT:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def NER_BERT(self, Word_Labels_List):
        embedded_input = []
        for named_entity in Word_Labels_List:
            inputs = self.tokenizer(named_entity, return_tensors="pt", padding=True, truncation=True)
            outputs = self.model(**inputs)
            last_hidden_states = outputs.last_hidden_state
            embedded_input.append(last_hidden_states.mean(dim=1).squeeze().detach().numpy())

        return embedded_input

In [13]:
ner_bert = NER_BERT()

C:\Users\gxb18167\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
embeddings = ner_bert.NER_BERT(train_NE)

In [16]:
len(embeddings[0])

2

In [22]:
embeddings[0].tolist()

[[-0.02876194380223751,
  -0.07249753922224045,
  -0.13805747032165527,
  -0.00011573234951356426,
  0.1949334293603897,
  -0.10386189073324203,
  0.3793443739414215,
  -0.20167122781276703,
  0.018948376178741455,
  -0.371090292930603,
  -0.06675172597169876,
  -0.06438987702131271,
  -0.05060451105237007,
  0.03611072897911072,
  -0.397544264793396,
  -0.12346602231264114,
  0.036294735968112946,
  -0.05520876124501228,
  -0.0650520995259285,
  0.28005847334861755,
  0.016437053680419922,
  -0.18223468959331512,
  0.11998198181390762,
  0.04991903901100159,
  0.004653478506952524,
  0.2526988089084625,
  -0.2539350986480713,
  -0.10897770524024963,
  -0.15387846529483795,
  0.16158972680568695,
  -0.1582179218530655,
  -0.11715751886367798,
  0.15636548399925232,
  0.40674689412117004,
  -0.19392995536327362,
  -0.2746075987815857,
  0.027851514518260956,
  0.023340387269854546,
  -0.33261892199516296,
  -0.11352729797363281,
  -0.01136713195592165,
  0.04025719687342644,
  0.1672230

In [46]:
vector_size = 768

In [24]:
expanded_named_entity_list = []
for i in range(len(train_EEG_segments)):
    named_entities = embeddings[i]
    for j in range(len(train_EEG_segments[i])):
        expanded_named_entity_list.append(named_entities)

In [60]:
max_seq_len = max([len(i) for i in expanded_named_entity_list])
#padding function
for i in range(len(expanded_named_entity_list)):
    padding_count = max_seq_len - len(expanded_named_entity_list[i])
    for j in range(padding_count):
        for z in range(len(expanded_named_entity_list)):
            expanded_named_entity_list[z] = np.concatenate([expanded_named_entity_list[z], np.zeros(vector_size,)], axis=0)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [51]:
type(expanded_named_entity_list)

list

In [45]:
desired_shape = (7, 768)
current_shape = expanded_named_entity_list[0].shape

In [46]:
pad_width = [(0, max(0, desired_shape[i] - current_shape[i])) for i in range(len(desired_shape))]

In [47]:
pad_width

[(0, 5), (0, 0)]

In [48]:
padded_arr = np.pad(expanded_named_entity_list[0], pad_width, mode='constant')

In [49]:
padded_arr.shape

(7, 768)

In [69]:
max_seq_len = max([len(i) for i in expanded_named_entity_list[0]])

In [70]:
max_seq_len

768

KeyboardInterrupt: 

In [42]:

max_seq_len = 0



for i in range(len(expanded_named_entity_list)):
    max_seq_len = max(max_seq_len, max_dimension_size(expanded_named_entity_list[i]))


In [43]:
max_seq_len

7